<a href="https://colab.research.google.com/github/hdpark1208/StudyCode/blob/main/10)%EA%B5%90%EC%B0%A8%EA%B2%80%EC%A6%9D%EA%B3%BC_%EA%B7%B8%EB%A6%AC%EB%93%9C%EC%84%9C%EC%B9%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***5-2 교차검증과 그리드 서치***

# **줄거리**

1.  검증(validation) 세트의 필요성을 이해하고 만들어본다  
(train_test_split을 두 번 실행)

2.  교차검증(cross validation)을 알아보고 cross_validate 를 이용한다  

3.  cross_validate 에서 분할기(splitter)의 필요성을 알고 이용한다  

4.  결정트리의 매개변수 값을 바꿔주며 가장 좋은 성능을 찾아보자 ~ 하이퍼파라미터(사용자 지정 파라미터) 튜닝  

5.  교차검증과 하이퍼파라미터 탐색을 한 번에 수행하는 그리드 서치를 배워본다  

6.  GridSearchCV클래스의 다양한 메서드를 알아본다

7.  params의 특성 수를 추가한 후 최상의 매개변수 조합을 확인해본다  

8.  (수동으로 매개변수를 바꾸는 대신)매개변수의 범위,간격을 담당하는 랜덤 서치를 알아본다  

9.  싸이파이(scipy) 라이브러리를 통해 확률 분포 객체를 이용한다  

#Note

*  교차검증의 최종 점수는 test_score 키에 담긴 5개의 점수(검증 폴드의 점수임)를 평균하여 얻는다  
*  cross_validate()는 훈련 세트를 섞어 폴드를 나누지않는다(앞에서 했던 train_test_split기능. 즉 주의하지않는다면 샘플링편향이 발생할 수 있음)  
*  랜덤 서치는 그리드 서치보다 훨씬 교차 검증 수를 줄이면서 넓은 영역을 효과적으로 탐색할 수 있다  


#**코드**


In [ ]:
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [ ]:
data = wine[['alcohol','sugar','pH']].to_numpy()
target = wine['class'].to_numpy()

In [ ]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target,test_target = train_test_split(data,target,test_size=0.2,random_state=42)

In [ ]:
sub_input,val_input,sub_target,val_target = train_test_split(train_input,train_target,test_size=0.2,random_state=42)

print(data.shape, target.shape)
print(train_input.shape, test_input.shape)
print(sub_input.shape, val_input.shape)

(6497, 3) (6497,)
(5197, 3) (1300, 3)
(4157, 3) (1040, 3)


In [ ]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state= 42)
dt.fit(sub_input,sub_target)
print(dt.score(sub_input,sub_target))
print(dt.score(val_input,val_target))


0.9971133028626413
0.864423076923077


In [ ]:
from sklearn.model_selection import cross_validate
scores = cross_validate(dt,train_input,train_target)
print(scores)

{'fit_time': array([0.00972557, 0.00713897, 0.00749874, 0.00737095, 0.00709176]), 'score_time': array([0.00086689, 0.00091505, 0.00073457, 0.00071049, 0.00067854]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [ ]:
import numpy as np
print(np.mean(scores['test_score']))

0.855300214703487


In [ ]:
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input,train_target,cv=StratifiedKFold())    #분류 모델일 경우 타켓 클래스를 골고루 나누기위해 StratifiedKFold 이용. 회귀 모델일 경우 KFold 분할기
print(np.mean(scores['test_score']))

0.855300214703487


In [ ]:
splitter = StratifiedKFold(n_splits=10, shuffle=True,random_state=42)    #n_splits=k : 몇(k) 폴드 교차 검증 할지
scores= cross_validate(dt,train_input,train_target,cv=splitter)
print(np.mean(scores['test_score']))


0.8574181117533719


In [ ]:
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease':[0.0001,0.0002,0.0003,0.0004,0.0005]}    #탐색할 매개변수와 탐색할 값의 리스트를 딕셔터리로 만든다

In [ ]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42),params,n_jobs=-1)    #탐색 대상 모델과 params 변수를 전달하여 그리드 서치 객체를 만든다

In [ ]:
gs.fit(train_input,train_target)    #검증 점수가 가장 높은 모델의 매개변수 조합으로 전체 훈련 세트에서 자동으로 다시 모델을 훈련한다

GridSearchCV(cv=None, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=42,
                                              splitter='best'),
             iid='deprecated', n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
    

In [ ]:
dt = gs.best_estimator_    
print(dt.score(train_input, train_target))

0.9615162593804117


In [ ]:
print(gs.best_params_)    #최적의 매개변수
print(gs.cv_results_['mean_test_score'])    #각 매개변수에서 수행한 교차 검증의 평균 점수

{'min_impurity_decrease': 0.0001}
[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [ ]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])    #np.argmax : 가장 큰 값의 인덱스 추출
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


In [ ]:
params = {'min_impurity_decrease':np.arange(0.0001,0.001,0.0001),
          'max_depth':range(5,20,1),    #range 는 정수만 취급
          'min_samples_split':range(2,100,10)}

In [ ]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42),params,n_jobs=-1)    #n_jobs : 매개변수에서 병렬 실행에 사용할 cpu 코어 수 (-1 : 모든 코어)
gs.fit(train_input,train_target)

GridSearchCV(cv=None, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=42,
                                              splitter='best'),
             iid='deprecated', n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'm

In [ ]:
print(gs.best_params_)
print(np.max(gs.cv_results_['mean_test_score']))

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}
0.8683865773302731


In [ ]:
from scipy.stats import uniform, randint

In [ ]:
rgen = randint(0,10)
rgen.rvs(5)

array([9, 8, 4, 8, 7])

In [ ]:
np.unique(rgen.rvs(1000),return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([110, 101,  93, 109,  99, 105,  77, 100, 108,  98]))

In [ ]:
ugen = uniform(0,1)
ugen.rvs(5)

array([0.08716302, 0.06818149, 0.89228799, 0.42689288, 0.52371536])

In [ ]:
params = {'min_impurity_decrease' : uniform(0.0001,0.001),
          'max_depth':randint(20,50),
          'min_samples_split':randint(2,25),
          'min_samples_leaf':randint(1,25)}

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42),params,
                        n_iter=100,n_jobs=-1,random_state=42)
gs.fit(train_input,train_target)

RandomizedSearchCV(cv=None, error_score=nan,
                   estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features=None,
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                                                    presort='deprecated',
                                                    random_state=42,
         

In [ ]:
print(gs.best_params_)
print(np.max(gs.cv_results_['mean_test_score']))

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}
0.8695428296438884


In [ ]:
dt=gs.best_estimator_
print(dt.score(test_input,test_target))

0.86
